<a href="https://colab.research.google.com/github/masoomehmashayekhi/AI-Exercises/blob/main/NLPTaskLastVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q pandas scikit-learn matplotlib
!pip install -q sentence-transformers
!pip install -q umap-learn
!pip install -q huggingface_hub

In [1]:
!pip install "hazm[full]" --no-deps && pip install numpy==1.26.4 fasttext-wheel flashtext gensim==4.3.3 nltk python-crfsuite

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Login using e.g. `huggingface-cli login` to access this dataset
splits = {'train': 'train.csv', 'validation': 'validation.csv', 'test': 'test.csv'}
df_train = pd.read_csv("hf://datasets/ParsiAI/snappfood-sentiment-analysis/" + splits["train"])
df_validation = pd.read_csv("hf://datasets/ParsiAI/snappfood-sentiment-analysis/" + splits["validation"])
df_test = pd.read_csv("hf://datasets/ParsiAI/snappfood-sentiment-analysis/" + splits["test"])

In [4]:
print("TrainSet Shape:" ,df_train.shape)
print("ValidationSet Shape:" ,df_validation.shape)
print("TestSet Shape:" ,df_test.shape)

TrainSet Shape: (52110, 3)
ValidationSet Shape: (8337, 3)
TestSet Shape: (9033, 3)


In [5]:
df_train.head()

,comment,label,label_id
0,غذا خیلی سرد بود در صورتیکه فاصله ما خیلی کم است,SAD,1.0
1,بهتره بتونیم ران یا سینه رو خودمون انتخاب کنیم,HAPPY,0.0
2,غذا بد بود حالم خیییییلی بده. دل دردو دل پیچه....,SAD,1.0
3,با سلام سابق بر این بسته بندی از کیفیت بهتری ب...,SAD,1.0
4,سلام، خیلی ممنون و متشکرم,HAPPY,0.0


In [6]:
df_train['label_id'].value_counts().sort_index()

,count
label_id,
0.0,26236
1.0,25874


In [7]:
print(df_train['comment'].isna().any())
print(df_train['label_id'].isna().any())

False
False


In [8]:
from hazm import *

normalizer = Normalizer()
df_train['comment_clean'] = df_train['comment'].astype(str).apply(normalizer.normalize)
df_validation['comment_clean'] = df_validation['comment'].astype(str).apply(normalizer.normalize)
df_test['comment_clean'] = df_test['comment'].astype(str).apply(normalizer.normalize)
df_train.head()

,comment,label,label_id,comment_clean
0,غذا خیلی سرد بود در صورتیکه فاصله ما خیلی کم است,SAD,1.0,غذا خیلی سرد بود در صورتیکه فاصله ما خیلی کم است
1,بهتره بتونیم ران یا سینه رو خودمون انتخاب کنیم,HAPPY,0.0,بهتره بتونیم ران یا سینه رو خودمون انتخاب کنیم
2,غذا بد بود حالم خیییییلی بده. دل دردو دل پیچه....,SAD,1.0,غذا بد بود حالم خیلی بده. دل دردو دل‌پیچه. معل...
3,با سلام سابق بر این بسته بندی از کیفیت بهتری ب...,SAD,1.0,با سلام سابق بر این بسته بندی از کیفیت بهتری ب...
4,سلام، خیلی ممنون و متشکرم,HAPPY,0.0,سلام، خیلی ممنون و متشکرم


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

tfidf_vectorixer= TfidfVectorizer(ngram_range=(1,2),
                                  min_df=5,
                                  max_df=0.9)

y_train=df_train['label_id']
y_test =df_test['label_id']
y_validation =df_validation['label_id']

X_train = tfidf_vectorixer.fit_transform(df_train['comment_clean'])
X_validation = tfidf_vectorixer.transform(df_validation['comment_clean'])
X_test = tfidf_vectorixer.transform(df_test['comment_clean'])
X_train.shape


(52110, 29611)

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

def accuracy(str,cl):
  y_pred = cl.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)

  # Precision
  precision = precision_score(y_test, y_pred, average='weighted')

  # Recall
  recall = recall_score(y_test, y_pred, average='weighted')

  # F1-score
  f1 = f1_score(y_test, y_pred, average='weighted')

  # Confusion matrix
  cm = confusion_matrix(y_test, y_pred)

  # گزارش کامل
  report = classification_report(y_test, y_pred)

  print(f"--------{str} scores-----")
  print(f"-------------------------------------")
  print("Accuracy :", accuracy)
  print("Precision:", precision)
  print("Recall   :", recall)
  print("F1-score :", f1)
  print("\nConfusion Matrix:\n", cm)
  print("\nClassification Report:\n", report)

In [11]:
from sklearn.linear_model import LogisticRegression


cl= LogisticRegression(max_iter=3000)
cl.fit(X_train,y_train)
accuracy("LogisticRegression",cl)

--------LogisticRegression scores-----
-------------------------------------
Accuracy : 0.8608435735636001
Precision: 0.8631162215360844
Recall   : 0.8608435735636001
F1-score : 0.8605805045767416

Confusion Matrix:
 [[3676  810]
 [ 447 4100]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.89      0.82      0.85      4486
         1.0       0.84      0.90      0.87      4547

    accuracy                           0.86      9033
   macro avg       0.86      0.86      0.86      9033
weighted avg       0.86      0.86      0.86      9033



In [12]:
from sklearn.naive_bayes import MultinomialNB


svm= MultinomialNB()
svm.fit(X_train,y_train)
accuracy("MultinomialNB",svm)

--------MultinomialNB scores-----
-------------------------------------
Accuracy : 0.8522085685818664
Precision: 0.8575323765904226
Recall   : 0.8522085685818664
F1-score : 0.8515821473203463

Confusion Matrix:
 [[3540  946]
 [ 389 4158]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.90      0.79      0.84      4486
         1.0       0.81      0.91      0.86      4547

    accuracy                           0.85      9033
   macro avg       0.86      0.85      0.85      9033
weighted avg       0.86      0.85      0.85      9033



In [13]:
from sklearn.ensemble import RandomForestClassifier


rf= RandomForestClassifier(n_estimators=300)
rf.fit(X_train,y_train)
accuracy("RandomForestClassifier",rf)

--------RandomForestClassifier scores-----
-------------------------------------
Accuracy : 0.8550869035757777
Precision: 0.8588603580712866
Recall   : 0.8550869035757777
F1-score : 0.8546439175628671

Confusion Matrix:
 [[3597  889]
 [ 420 4127]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.90      0.80      0.85      4486
         1.0       0.82      0.91      0.86      4547

    accuracy                           0.86      9033
   macro avg       0.86      0.85      0.85      9033
weighted avg       0.86      0.86      0.85      9033



In [22]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

print(tf.config.list_physical_devices('GPU'))

tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df_train['comment_clean'])

sequences = tokenizer.texts_to_sequences(df_train['comment_clean'])
X_train = pad_sequences(sequences, maxlen=50, padding='post', truncating='post')

y_train = to_categorical(df_train['label_id'])


sequences_valid = tokenizer.texts_to_sequences(df_validation['comment_clean'])
X_valid = pad_sequences(sequences_valid, maxlen=50, padding='post', truncating='post')

y_validation = to_categorical(df_validation['label_id'])


sequences_test = tokenizer.texts_to_sequences(df_test['comment_clean'])
X_test = pad_sequences(sequences_test, maxlen=50, padding='post', truncating='post')

y_test = to_categorical(df_test['label_id'])

model = Sequential([
    Embedding(5000, 128, input_length=50),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dropout(0.5),
    Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    Dense(2, activation='softmax')
])

es = EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True
)

model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

model.build(input_shape=(None, 50))
model.summary()


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 50, 128)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_10                │ (None, 50, 128)        │        98,816 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 50, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_11                │ (None, 64)             │        41,216 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 782,178 (2.98 MB)

 Trainable params: 782,178 (2.98 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:

history = model.fit(X_train, y_train, epochs=10, batch_size=16,validation_data=(X_valid, y_validation), verbose=1,callbacks=[es])

loss, acc = model.evaluate(X_test, y_test)

Epoch 1/10
3257/3257 ━━━━━━━━━━━━━━━━━━━━ 57s 16ms/step - accuracy: 0.8081 - loss: 0.4345 - val_accuracy: 0.8586 - val_loss: 0.3368
Epoch 2/10
3257/3257 ━━━━━━━━━━━━━━━━━━━━ 53s 16ms/step - accuracy: 0.8746 - loss: 0.3040 - val_accuracy: 0.8591 - val_loss: 0.3453
Epoch 3/10
3257/3257 ━━━━━━━━━━━━━━━━━━━━ 52s 16ms/step - accuracy: 0.8879 - loss: 0.2754 - val_accuracy: 0.8577 - val_loss: 0.3430
283/283 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8590 - loss: 0.3362


In [24]:
y_pred_rnn = model.predict(X_test)
y_test_prob = np.argmax(y_test, axis=1)


y_pred_rnn_prob = np.argmax(y_pred_rnn, axis=1)

acc_rnn = accuracy_score(y_test_prob, y_pred_rnn_prob)
p_rnn, r_rnn, f1_rnn, _ = precision_recall_fscore_support(y_test_prob, y_pred_rnn_prob, average='weighted')


cm_rnn = confusion_matrix(y_test_prob, y_pred_rnn_prob)

283/283 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


In [25]:
print(f"--------RNN scores-----")
print(f"---------------------")
print("Accuracy :", acc_rnn)
print("Precision:", p_rnn)
print("Recall   :", r_rnn)
print("F1-score :", f1_rnn)
print("\nConfusion Matrix:\n", cm_rnn)

--------RNN scores-----
---------------------
Accuracy : 0.8587401749142035
Precision: 0.861140344178143
Recall   : 0.8587401749142035
F1-score : 0.8584578664929918

Confusion Matrix:
 [[3661  825]
 [ 451 4096]]


In [26]:
from datasets import Dataset

label2id = {"negative": 0, "positive": 1}
id2label = {0: "negative", 1: "positive"}

dataset = Dataset.from_pandas(df_train)
datatest = Dataset.from_pandas(df_test)

dataset

Dataset({
    features: ['comment', 'label', 'label_id', 'comment_clean'],
    num_rows: 52110
})

In [27]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)


model_name = "HooshvareLab/bert-fa-zwnj-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

tokenizer_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/473M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
def preprocess_function(examples):
    return tokenizer(examples["comment_clean"], truncation=True, padding=True, max_length=128)

tokenized_dataset_train = dataset.map(preprocess_function, batched=True)
tokenized_dataset_test = datatest.map(preprocess_function, batched=True)


Map:   0%|          | 0/52110 [00:00<?, ? examples/s]

Map:   0%|          | 0/9033 [00:00<?, ? examples/s]

In [29]:
tokenized_train = tokenized_dataset_train.map(lambda x: {"labels": int(x["label_id"])})
tokenized_test  = tokenized_dataset_test.map(lambda x: {"labels": int(x["label_id"])})

Map:   0%|          | 0/52110 [00:00<?, ? examples/s]

Map:   0%|          | 0/9033 [00:00<?, ? examples/s]

In [30]:
cols_to_remove = ['comment', 'comment_clean', 'label', 'label_id']

tokenized_train = tokenized_train.remove_columns([c for c in cols_to_remove if c in tokenized_train.column_names])
tokenized_test  = tokenized_test.remove_columns([c for c in cols_to_remove if c in tokenized_test.column_names])

In [31]:

tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_train

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 52110
})

In [32]:
tokenized_train.column_names
tokenized_train[0]

{'input_ids': tensor([    2,  4150,  2712,  3657,  1976,  1921, 15944,  3402,  2121,  2712,
          2067,  1933,     3,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [33]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    p, r, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {"accuracy": acc, "f1": f1, "precision": p, "recall": r}

In [39]:
from transformers import DataCollatorWithPadding, EarlyStoppingCallback
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./sentiment_farsi_model",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=True,
    dataloader_num_workers=4,
    report_to="none",
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

/tmp/ipython-input-639695121.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [40]:
import torch
print("GPU available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

GPU available: True
Device: Tesla T4


In [41]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.266100,0.400637,0.852209,0.852147,0.852600,0.852209
2,0.270200,0.373833,0.862283,0.861982,0.864954,0.862283
3,0.357400,0.409169,0.858297,0.858017,0.860668,0.858297
4,0.232400,0.401808,0.857412,0.856656,0.864290,0.857412


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: T

TrainOutput(global_step=13028, training_loss=0.3407688363402908, metrics={'train_runtime': 1853.0895, 'train_samples_per_second': 140.603, 'train_steps_per_second': 8.788, 'total_flos': 1.37107170948096e+16, 'train_loss': 0.3407688363402908, 'epoch': 4.0})

In [42]:
trainer.evaluate()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


{'eval_loss': 0.37383270263671875,
 'eval_accuracy': 0.8622827410605557,
 'eval_f1': 0.8619822702898232,
 'eval_precision': 0.8649544797792477,
 'eval_recall': 0.8622827410605557,
 'eval_runtime': 14.2244,
 'eval_samples_per_second': 635.036,
 'eval_steps_per_second': 19.895,
 'epoch': 4.0}